In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim

import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Value, Case, When, IntegerField

In [13]:
def clean_text(text):
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    text = text.replace(u'\xa0', ' ')
    text = text.replace(u'\x96', '-')
    text = text.replace(u'\xad', '-')
    text = text.replace(u'\u2014', '–')
    # text = text.replace(u'\u2013', '–')
    text = text.replace('(', '')
    text = text.replace(')', '')
    return text

### Finding agenda items

Search id 89: Searches 84, 85, 88, 87 combined 
- 84: (01/01 - 13/85) updated - from https.*scans of pdfs with xml metadata
- 85: (13/86 - 18/210) GermaParlTEI
- 87: (19/01 - 19/108) updated - XML from www.bundestag.de/service/opendata
- 88: (18/211 - 18/245) from https.*scans of pdfs with xml metadata

Search id 96: Searches 90, 95, 92, 93 combined
- 90: (01/01 - 13/85) updated - from https.*scans of pdfs with xml metadata
- 92: (18/211 - 18/245) from https.*scans of pdfs with xml metadata
- 93: (19/01 - 19/108) updated - XML from www.bundestag.de/service/opendata
- 95: (13/86 - 18/210) updated - GermaParlTEI

In [3]:
# finding the agenda items of the utterances that appear in the search
utterances = pm.Utterance.objects.filter(search_matches__pk=103)
#agenda_items = pm.AgendaItem.objects.filter()

In [4]:
agenda_items = []
agenda_items_doc_date = []
count_missing = 0
missing_doc_date = []
for ut in utterances:
    try:
        agenda_items.append(ut.agenda_item.title)
        agenda_items_doc_date.append(ut.document.date)
    except AttributeError:
        count_missing += 1
        missing_doc_date.append(ut.document.date)
        pass

print(len(agenda_items))
print(count_missing)

7866
1275


In [8]:
missing_doc_date

[datetime.date(2018, 5, 16),
 datetime.date(2018, 1, 31),
 datetime.date(2018, 1, 31),
 datetime.date(2018, 1, 31),
 datetime.date(2018, 5, 16),
 datetime.date(2018, 5, 16),
 datetime.date(2018, 5, 18),
 datetime.date(2018, 6, 13),
 datetime.date(2018, 6, 13),
 datetime.date(2019, 2, 13),
 datetime.date(2019, 2, 13),
 datetime.date(2019, 4, 10),
 datetime.date(2019, 4, 10),
 datetime.date(2019, 4, 10),
 datetime.date(2019, 4, 10),
 datetime.date(2018, 9, 12),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 6, 26),
 datetime.date(2019, 5, 8),
 datetime.date(2019, 5, 8),
 datetime.date(2019, 5, 8),
 datetime.date(2019, 5, 8),
 datetime.date(2019, 6, 5),
 datetime.date(2018, 9, 14),
 datetime.date(1995, 3, 29),
 datetime.date(1995, 3, 29),
 datetime.date(1995, 3, 29),
 datetime.date(1995, 3, 29),
 datetime.date(1995, 3, 29),
 datetime.date(1995, 3, 29),
 datetime.date(1995, 3, 29),
 datetime.date(1995

In [5]:
df_agenda_items = pd.DataFrame(agenda_items)
df_agenda_items.columns = ['Agenda Item']
d = {'Agenda Item': agenda_items, 'Date': agenda_items_doc_date}
df_d = pd.DataFrame.from_dict(d)

In [70]:
# easier way to get the data
ai = pm.Utterance.objects.filter(search_matches__pk=103).values('agenda_item__title', 'document__date', 'id')
ai_df = pd.DataFrame.from_dict(ai).dropna()
ai_df.columns = pd.Index(['Agenda Item', 'Date', 'ID'])

In [74]:
# extract unique agenda items only
# unique by Agenda Item label
df_agenda_items_unique = ai_df.drop_duplicates(subset=['Agenda Item']).sort_values(by=['Date'])
# unique by date and label
#df_agenda_items_unique = ai_df.drop_duplicates(subset=['Agenda Item', 'Date']).sort_values(by=['Date'])

pd.set_option('max_colwidth', 1000)
print(df_agenda_items_unique.count())
df_agenda_items_unique

Agenda Item    2222
Date           2222
ID             2222
dtype: int64


,Agenda Item,Date,ID
8215,"Präsident Dr. Köhler: Meine Damen und Herren! Inzwischen. ist von den verschiedensten Seiten des Hauses der Wunsch an mich herangetragen worden, etwa gegen ein Uhr eine kürzere Mittagspause zu machen und bis dahin möglichst noch die Begründung der Anträge zu den Tagesordnungspunkten\n3. Antrag der Fraktion der KPD betr. sofortige Einstellung der Demontagen (Drucksache Nr. 6),",1949-09-30,4495145
8221,"Ich stelle fest: das ist nicht der Fall. Dann behandeln wir gemeinsam die Tagesordnungspunkte 14, 15, 16, 17 und 23.\nWir kommen zunächst zu Punkt 14 der Tagesordnung:",1949-10-20,4495355
8220,"Ich rufe auf die Punkte 10, 11 und 12 der Tagesordnung:\nAntrag der Fraktion der KPD, betreffend sozialen Wohnungsbau (Drucksache Nr. 10);",1949-10-20,4495314
8225,"Ich rufe also den Punkt 2 der Tagesordnung auf: Erste Beratung des Entwurfs eines Gesetzes über die Erstreckung der bei den Annahmestellen Darmstadt und Berlin eingereichten Patent-, Gebrauchsmuster- und Warenzeichenanmeldungen auf die Länder Baden, Rheinland-Pfalz, Württemberg-Hohenzollern und den bayerischen Kreis Lindau (Drucksache Nr. 152).\nWelcher der Herren Minister wünscht die Einbringung der Vorlage zu begründen? – Das Wort hat der Herr Bundesminister der Justiz.",1949-11-10,4495556
8239,"Sie nimmt diesen Abgeordneten aas Recht, ihre Meinung, das heißt die Meinung der Wähler, die hinter den Abgeordneten stehen, hier zu den jeweiligen Tagesordnungspunkten zu sagen. Wie man aus dem Protest gegen derartige Methoden die Absicht der Sabotage der Arbeit des Parlaments konstruieren kann, das ist mir schleierhaft. Das Parlament wird in seinem Wert draußen nach der praktischen Arbeit beurteilt, die hier geleistet wird.\n(Zuruf: Das ist richtig! – Weitere Zurufe.)",1950-01-11,4496389
8241,Ich rufe auf Punkt 3 der Tagesordnung.\n(Zuruf rechts: Abgesetzt!),1950-01-20,4496672
8245,Ich rufe auf Punkt 10 der Tagesordnung: Beratung des Antrags der Fraktion der KPD betreffend abgelaufenes Produktionspermit (Drucksache Nr. 412).\nDas Wort hat der Herr Bundesarbeitsminister.,1950-02-01,4496927
8258,"Präsident Dr. Köhler: Meine Damen und Herren! Ich eröffne erneut die heute mittag vertagte 37. Sitzung des Deutschen Bundestags mit dem einzigen Tagesordnungspunkt 2:\nBeratung des Mündlichen Berichts des Ausschusses für Wirtschaftspolitik über die Anträge der Fraktion der WAV betreffend Benzinpreiserhöhung, der Fraktion der KPD betreffend Mißbilligung der Anordnung des Bundesministers für Wirtschaft auf Erhöhung der Mineralölpreise und Antrag auf Aufhebung derselben, der Abgeordneten Rademacher, Stahl, Dr. Oellers, Dr. Schäfer, Dr. Wellhausen und Fraktion der FDP betreffend Preiserhöhung für Treibstoff (Drucksachen Nr. 501, 465, 363, 384).",1950-02-10,4497282
8262,Vizepräsident Dr. Schmid: Ich rufe auf Punkt 6 der Tagesordnung:\nInterfraktioneller Antrag betreffend Überweisung von Anträgen an die Ausschüsse (Drucksache Nr. 533).,1950-02-16,4497466
8275,Ich rufe also nunmehr den eben zurückgestellten Punkt 11 der Tagesordnung auf:\nBeratung des Mündlichen Berichts des,1950-03-01,4497875


---

In [14]:
agenda_items_unique_lower = [word.lower() for word in agenda_items_unique]
agenda_items_unique_clean = [clean_text(text) for text in agenda_items_unique_lower]

# show simple word counts

In [30]:
import urllib.request
from collections import Counter

import numpy as np

from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

data = " ".join(agenda_items_unique_clean)

stopword_list = stopwords.words("german")

# Note that `ngram_range=(1, 1)` means we want to extract Unigrams, i.e. tokens.
ngram_vectorizer = CountVectorizer(analyzer='word', tokenizer=word_tokenize,
                                   ngram_range=(1, 1), min_df=1, stop_words=stopword_list)
# X matrix where the row represents sentences and column is our one-hot vector for each token in our vocabulary
X = ngram_vectorizer.fit_transform(data.split('\n'))

# Vocabulary
vocab = list(ngram_vectorizer.get_feature_names())

# Column-wise sum of the X matrix.
# It's some crazy numpy syntax that looks horribly unpythonic
# For details, see http://stackoverflow.com/questions/3337301/numpy-matrix-to-array
# and http://stackoverflow.com/questions/13567345/how-to-calculate-the-sum-of-all-columns-of-a-2d-numpy-array-efficiently
counts = X.sum(axis=0).A1

freq_distribution = Counter(dict(zip(vocab, counts)))
for item in freq_distribution.most_common(200):
    print(item)

(',', 4056)
('.', 2227)
(':', 1511)
('beratung', 1433)
('rufe', 1121)
('tagesordnung', 909)
('punkt', 777)
('dr.', 757)
('–', 746)
('a', 576)
('bundesregierung', 503)
('abgeordneten', 480)
('...', 468)
('entwurfs', 466)
('tagesordnungspunkt', 456)
('gesetzes', 452)
('antrags', 403)
('fraktion', 399)
('eingebrachten', 396)
('spd', 286)
('1', 285)
('drucksache', 274)
('herren', 265)
('zweite', 264)
('damen', 258)
('erste', 256)
('2', 254)
('dritte', 234)
('wort', 217)
('aussprache', 210)
('drucksachen', 201)
('herr', 201)
('cdu/csu', 200)
('!', 194)
('fragestunde', 193)
(';', 187)
('3', 186)
('ausschusses', 174)
('tagesordnungspunkte', 172)
('berichts', 162)
('frau', 161)
('nr', 153)
('?', 151)
('fraktionen', 151)
('änderung', 151)
('4', 148)
('ausschuß', 144)
('5', 128)
('zusatzpunkt', 123)
('feststellung', 118)
('fdp', 116)
('beschlossen', 114)
('abgeordneter', 113)
('antrag', 113)
('anfrage', 112)
('widerspruch', 105)
('bundeshaushaltsplans', 102)
('abgabe', 99)
('ältestenrat', 98)
('

In [81]:
# count number of keyword occurrances (regex)
keywords = ['haushalt', 'kohle', 'wirtschaft', 'energie', 'strom', 'atom', 'ausstieg', 'klima']
print("keyword \tcounter\t sum of occurrances")
for keyword in keywords:
    SEARCH = re.compile(keyword, re.IGNORECASE)

    occurrances_re = np.zeros(len(agenda_items_unique_clean))
    counter = 0
    for i, text in enumerate(agenda_items_unique_clean):
        matches = SEARCH.findall(text)
        occurrances_re[i] = len(matches)
        if len(matches) > 0:
            counter += 1

    print(keyword, "   \t", counter, "\t", occurrances_re.sum())

keyword 	counter	 sum of occurrances
haushalt    	 167 	 351.0
kohle    	 41 	 58.0
wirtschaft    	 152 	 185.0
energie    	 52 	 78.0
strom    	 12 	 13.0
atom    	 20 	 24.0
ausstieg    	 1 	 1.0
klima    	 10 	 12.0


In [82]:
# show all agenda items with specific keyword

selection = df_agenda_items_unique[df_agenda_items_unique['Agenda Item'].str.contains('klima', case=False)].sort_values(by=['Date'])
print(selection.count())
selection

Agenda Item    10
Date           10
ID             10
dtype: int64


,Agenda Item,Date,ID
5659,"Dr. Luck, der Leiter der Forschungsstelle, hat bei dem Besuch Themen angesprochen, die gerade diesen Tagesordnungspunkt berühren. Dr. Luck räumt ein, daß Belastungen der Nordsee nicht zu leugnen sind. Er unterstreicht, daß die Schadstoffeinträge reduziert werden müssen. Er stellt aber auch fest, daß die Aussage, „wenn nicht sofort etwas passiere, werde die Nordsee umkippen"", über das Ziel hinausgehe.\nDr. Luck stellt allerdings nachdrücklich fest – das ist ein ernstes Thema, das wir vielleicht ein bißchen beleuchten sollten –, daß in der öffentlichen Diskussion die Klimaveränderungen nicht berücksichtigt werden und somit die weltweit festzustellende Beschleunigung des Anstiegs des Meeresspiegels nicht ausreichend gewürdigt wird. Wenn der Meeresspiegel im Atlantik stiege, bedeutete das für die Nordsee bis zur Jahrhundertwende einen Anstieg des Meeresspiegels um einen Meter. Schon jetzt bringt Dr. Luck Dünenabbrüche bei der Insel Langeoog sowie Veränderungen am Westende der Insel Jui...",1986-06-05,4151263
5372,"Müller (Düsseldorf) (SPD): Frau Präsidentin! Meine Damen und Herren! Frau Segall, der Tagesordnungspunkt heißt insgesamt „Bericht der Enquete-Kommission"". Insofern werde ich in Anspruch nehmen, hier über die Klimaproblematik zu reden. Frau Ganseforth redet über die Ozonproblematik; nur damit Sie sich darüber im klaren sind.\nDer Bericht der Enquete-Kommission belegt, daß wir mit dem fortgesetzten Atmosphärenkrieg der Menschheit die Erde auf eine globale Umweltkrise zusteuern. Diese besteht in zweierlei Hinsicht:",1989-03-09,4073124
3939,"Dr. Klaus Töpfer, Bundesminister für Umwelt, Naturschutz und Reaktorsicherheit: Frau Präsidentin! Meine sehr verehrten Damen und Herren! In den angelsächsischen Ländern, insbesondere in den Vereinigten Staaten, ist heute „Earthday"", Tag der Erde, der Umwelttag, wie wir ihn im Juni haben. Es ist vielleicht gut, darauf hinzuweisen. Denn die heutige Tagesordnung knüpft hier unmittelbar an. Wir behandeln im ersten Tagesordnungspunkt den globalen Umweltschutz. Wir werden uns danach mit dem kontinentalen, mit dem europäischen Umweltschutz beschäftigen und hinterher Fragen des Naturschutzes und der Umwelt im nationalen Bereich erörtern. Das zeigt den Spannungsrahmen. Ich finde, man sollte einen solchen guten Zusammenhang in der Tagesordnung einmal ansprechen.\nGlobal, kontinental, national, bis zum einzelnen: Das ist der Rahmen, in dem wir Umweltpolitik gestalten müssen. Ich glaube, daß wir das in ganz besonderer Weise auch bezüglich der Gesetze tun können, die wir heute in erster Lesung ...",1993-04-22,4021402
3955,Ich rufe den Tagesordnungspunkt 7 auf:\na) Zweite Beratung und Schlußabstimmung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes zu dem Rahmenübereinkommen der Vereinten Nationen vom 12. Juni 1992 über Klimaänderungen,1993-06-23,4024598
3551,"Ich rufe jetzt Tagesordnungspunkt 11 und Zusatzpunkt 9 auf:\n11. Beratung des Schlußberichts der Enquete-Kommission „Schutz der Erdatmosphäre"" zum Thema Mehr Zukunft für die Erde – Nachhaltige Energiepolitik für dauerhaften Klimaschutz -",1995-01-20,3968857
3494,Ich rufe die Tagesordnungspunkte 1 a bis 1 c und Zusatzpunkt 1 auf:\n1. a) Abgabe einer Erklärung der Bundesregierung zu den Ergebnissen der Berliner Klimakonferenz sowie zu aktuellen Fragen der Kernenergie,1995-04-26,3973824
3619,"Dr. Renate Hellwig (CDU/CSU): Herr Präsident! Meine sehr verehrten Damen und Herren! Das eigentliche und Hauptthema des Tagesordnungspunktes 1 ist nach wie vor die Klimakonferenz. Ich möchte daran erinnern und darauf zurückkommen, aber gleichzeitig auch klarmachen, in welch engem Zusammenhang die Klimakonferenz mit der Frage der Kernenergie steht.\nWir haben - das ist zu Recht heute hier noch einmal deutlich gemacht worden - auf dieser Klimakonferenz weltweit ein Exempel statuiert, indem unser Bundeskanzler für Deutschland die klare Zielvorgabe gegeben hat, daß von 1990 bis 2005 - wir sind bereits im Jahre 1995 

In [79]:
# show all speeches with keyword in agenda item
selection = ai_df[ai_df['Agenda Item'].str.contains('kohle', case=False)].sort_values(by=['Date'])
selection

,Agenda Item,Date,ID
8344,"Dann rufe ich auf Punkt 13 der Tagesordnung: Beratung des Antrages der Fraktion der SPD betreffend Eigentumsregelung in der Kohlen-, Eisen- und Stahlwirtschaft (Nr. 1549 der Drucksachen).\n(Abg. Mellies: Herr Präsident, es ist gesagt",1950-12-07,4503135
8865,Ich rufe auf Punkt 8 der Tagesordnung:\nBeratung des Antrags der Fraktion der SPD betreffend Kohlenförderung im Warndt (Nr. 3023 der Drucksachen).,1951-02-21,4512774
8864,Ich rufe auf Punkt 8 der Tagesordnung:\nBeratung des Antrags der Fraktion der SPD betreffend Kohlenförderung im Warndt (Nr. 3023 der Drucksachen).,1951-02-21,4512773
9124,"Ich rufe auf Punkt 2 der Tagesordnung: Beratung des Entwurfs einer Verordnung PR Nr. 12/51 zur Verlängerung der Geltungsdauer der Verordnung PR Nr. 79/50 zur Änderung von Preisen für Steinkohle, Steinkohlenkoks und Steinkohlenbriketts aus den Revieren Ruhr und Aachen (Nr. 2037 der Drucksachen).\nDie Regierung hat mir mitgeteilt, daß sie sich auf die schriftliche Begründung bezieht.",1951-03-16,4505261
9121,"Ich rufe auf Punkt 2 der Tagesordnung: Beratung des Entwurfs einer Verordnung PR Nr. 12/51 zur Verlängerung der Geltungsdauer der Verordnung PR Nr. 79/50 zur Änderung von Preisen für Steinkohle, Steinkohlenkoks und Steinkohlenbriketts aus den Revieren Ruhr und Aachen (Nr. 2037 der Drucksachen).\nDie Regierung hat mir mitgeteilt, daß sie sich auf die schriftliche Begründung bezieht.",1951-03-16,4505257
9120,"Ich rufe auf Punkt 2 der Tagesordnung: Beratung des Entwurfs einer Verordnung PR Nr. 12/51 zur Verlängerung der Geltungsdauer der Verordnung PR Nr. 79/50 zur Änderung von Preisen für Steinkohle, Steinkohlenkoks und Steinkohlenbriketts aus den Revieren Ruhr und Aachen (Nr. 2037 der Drucksachen).\nDie Regierung hat mir mitgeteilt, daß sie sich auf die schriftliche Begründung bezieht.",1951-03-16,4505256
9123,"Ich rufe auf Punkt 2 der Tagesordnung: Beratung des Entwurfs einer Verordnung PR Nr. 12/51 zur Verlängerung der Geltungsdauer der Verordnung PR Nr. 79/50 zur Änderung von Preisen für Steinkohle, Steinkohlenkoks und Steinkohlenbriketts aus den Revieren Ruhr und Aachen (Nr. 2037 der Drucksachen).\nDie Regierung hat mir mitgeteilt, daß sie sich auf die schriftliche Begründung bezieht.",1951-03-16,4505259
8625,Ich rufe auf Punkt 10 der Tagesordnung:\nBeratung des Antrags der Fraktion der Bayernpartei betreffend Anrechnung von Besatzungskohle auf die Exportquote (Nr. 2170 der Drucksachen).,1951-05-09,4506808
8626,Ich rufe auf Punkt 10 der Tagesordnung:\nBeratung des Antrags der Fraktion der Bayernpartei betreffend Anrechnung von Besatzungskohle auf die Exportquote (Nr. 2170 der Drucksachen).,1951-05-09,4506809
8642,Ich rufe Punkt 1 der Tagesordnung auf:\nBeratung des Antrags der Fraktion der SPD betreffend Neuordnung der Eisen- und Stahlindustrie und des Kohlenbergbaues (Nr. 2264 der Drucksachen).,1951-06-06,4507310


---

### Preparing corpus and dictionary

In [14]:
# prepare corpus for topic model
from gensim import corpora
from nltk.corpus import stopwords
stop_words = set(stopwords.words('german'))

# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in agenda_items_unique_clean]

# Remove stopwords 
texts_no_sw = []
for text in texts:
    no_sw = [word for word in text if word not in stop_words]
    texts_no_sw.append(no_sw)

# Create dictionary
dictionary = corpora.Dictionary(texts_no_sw)

# Get information about the dictionary
print(dictionary)

#Convert document into the bag-of-words (BoW) format = list of (token_id, token_count) tuples.
agenda_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts_no_sw]

### Running a simple topic model

In [12]:
# Step 0: Import packages and stopwords
from gensim.models import LdaModel, LdaMulticore
from gensim.utils import simple_preprocess, lemmatize
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)

In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=agenda_corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

2019-08-11 16:45:14,305 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2019-08-11 16:45:14,308 : INFO : using symmetric eta at 0.1
2019-08-11 16:45:14,309 : INFO : using serial LDA version on this node
2019-08-11 16:45:14,318 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 99 documents, updating model once every 99 documents, evaluating perplexity every 99 documents, iterating 50x with a convergence threshold of 0.001000
2019-08-11 16:45:14,469 : INFO : -10.503 per-word bound, 1451.4 perplexity estimate based on a held-out corpus of 99 documents with 1214 words
2019-08-11 16:45:14,470 : INFO : PROGRESS: pass 0, at document #99/99
2019-08-11 16:45:14,591 : INFO : optimized alpha [0.09454434, 0.09445321, 0.09212606, 0.078220725, 0.080531895, 0.07877312, 0.07648808, 0.07261585, 0.08237724, 0.076027945]
2019-08-11 16:45:14,597 : INFO : topic #7 (0.073): 0.032*"beratung" + 0.017*"cdu/csu" +

2019-08-11 16:45:15,067 : INFO : topic #1 (0.082): 0.055*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + 0.051*"abgeordneten" + 0.051*"weiterer" + 0.041*"antrags" + 0.031*"afd" + 0.028*"b" + 0.019*"fdp"
2019-08-11 16:45:15,068 : INFO : topic #0 (0.085): 0.058*"fraktion" + 0.046*"grünen" + 0.046*"90/die" + 0.046*"bündnis" + 0.044*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + 0.040*"abgeordneten" + 0.030*"antrags"
2019-08-11 16:45:15,069 : INFO : topic diff=0.055881, rho=0.408248
2019-08-11 16:45:15,131 : INFO : -5.663 per-word bound, 50.7 perplexity estimate based on a held-out corpus of 99 documents with 1214 words
2019-08-11 16:45:15,132 : INFO : PROGRESS: pass 5, at document #99/99
2019-08-11 16:45:15,172 : INFO : optimized alpha [0.08382252, 0.08033282, 0.07262361, 0.05606041, 0.057044327, 0.058720324, 0.05190908, 0.044839364, 0.05767987, 0.049393546]
2019-08-11 16:45:15,174 : INFO : topic #7 (0.045): 0.019*"beratung" + 0.018*"cdu/csu" + 0

2019-08-11 16:45:15,688 : INFO : topic #1 (0.074): 0.056*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + 0.051*"abgeordneten" + 0.051*"weiterer" + 0.042*"antrags" + 0.032*"afd" + 0.028*"b" + 0.019*"fdp"
2019-08-11 16:45:15,690 : INFO : topic #0 (0.079): 0.059*"fraktion" + 0.048*"grünen" + 0.048*"90/die" + 0.048*"bündnis" + 0.043*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + 0.039*"abgeordneten" + 0.030*"antrags"
2019-08-11 16:45:15,691 : INFO : topic diff=0.016337, rho=0.301511


In [14]:
doc_lda = lda_model[agenda_corpus]

In [15]:
import pprint
pp = pprint.PrettyPrinter(indent=0)
pp.pprint(lda_model.print_topics())

2019-08-11 16:45:16,983 : INFO : topic #0 (0.079): 0.059*"fraktion" + 0.048*"grünen" + 0.048*"90/die" + 0.048*"bündnis" + 0.043*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + 0.039*"abgeordneten" + 0.030*"antrags"
2019-08-11 16:45:16,985 : INFO : topic #1 (0.074): 0.056*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + 0.051*"abgeordneten" + 0.051*"weiterer" + 0.042*"antrags" + 0.032*"afd" + 0.028*"b" + 0.019*"fdp"
2019-08-11 16:45:16,986 : INFO : topic #2 (0.065): 0.049*"abgeordneten" + 0.046*"beratung" + 0.046*"fraktion" + 0.041*"weiterer" + 0.039*"antrags" + 0.037*"abgeordneter" + 0.023*"dr." + 0.019*"linke" + 0.016*"grünen" + 0.016*"90/die"
2019-08-11 16:45:16,987 : INFO : topic #3 (0.049): 0.076*"bundesregierung" + 0.072*"beratung" + 0.047*"erste" + 0.047*"entwurfs" + 0.047*"eingebrachten" + 0.047*"gesetzes" + 0.020*"unterrichtung" + 0.011*"berichts" + 0.011*"beschlussempfehlung" + 0.010*"ausschusses"
2019-08-11 16:45:16,988 : INFO : topic 

[(0,
'0.059*"fraktion" + 0.048*"grünen" + 0.048*"90/die" + 0.048*"bündnis" + '
'0.043*"dr." + 0.043*"beratung" + 0.040*"abgeordneter" + 0.040*"weiterer" + '
'0.039*"abgeordneten" + 0.030*"antrags"'),
(1,
'0.056*"beratung" + 0.054*"dr." + 0.054*"fraktion" + 0.051*"abgeordneter" + '
'0.051*"abgeordneten" + 0.051*"weiterer" + 0.042*"antrags" + 0.032*"afd" + '
'0.028*"b" + 0.019*"fdp"'),
(2,
'0.049*"abgeordneten" + 0.046*"beratung" + 0.046*"fraktion" + 0.041*"weiterer" '
'+ 0.039*"antrags" + 0.037*"abgeordneter" + 0.023*"dr." + 0.019*"linke" + '
'0.016*"grünen" + 0.016*"90/die"'),
(3,
'0.076*"bundesregierung" + 0.072*"beratung" + 0.047*"erste" + 0.047*"entwurfs" '
'+ 0.047*"eingebrachten" + 0.047*"gesetzes" + 0.020*"unterrichtung" + '
'0.011*"berichts" + 0.011*"beschlussempfehlung" + 0.010*"ausschusses"'),
(4,
'0.050*"abgeordneten" + 0.043*"fraktion" + 0.040*"beratung" + '
'0.037*"abgeordneter" + 0.037*"weiterer" + 0.031*"antrags" + 0.022*"linke" + '
'0.020*"fdp" + 0.013*"beer," + 0.013*"n

### Visualising results of topic model

In [16]:
import pyLDAvis 
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, agenda_corpus, dictionary)
vis

ModuleNotFoundError: No module named 'pyLDAvis'